In [1]:
! pip install docx2txt
! pip install pypdf
! pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=03aa28fcada814b1ae33ea4e012afd06cc060952d05b260857237b999022962d
  Stored in directory: /Users/songyuze/Library/Caches/pip/wheels/55/f0/2c/81637d42670985178b77df6d41b9b6c6dc18c94818447414b9
Successfully built docx2txt


In [18]:
! pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.7/195.7 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 24.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 14.8

In [1]:
import common.utils
import os
import openai
openai.organization = ''
openai.api_key = common.utils.get_openai_api_key()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/songyuze/opt/anaconda3/envs/RAG/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 3


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from langchain.document_loaders import UnstructuredExcelLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# define chatdoc
class ChatDoc():
    def __init__(self):
        self.doc = None
        self.splitText = []
        self.template = [
            ("system", "You are an assistant to process the documents, you don't speak you are an AI developed by OpenAI, you will continue answer the question based on the content I provided to you.\n content: \n {context} \n"),
            ("human", "Hello!"),
            ("ai", "Hello!"),
            ("human", "{question}")
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

    def getFile(self):
        doc = self.doc
        loaders = {
            "docx": Docx2txtLoader,
            "pdf": PyPDFLoader,
            "xlsx": UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e:
                print(f"Error loading {file_extension} files: {e}")
                return None
        else:
            raise Exception("Unsupported file type")
            return None
    
    def splitSentences(self):
        full_text = self.getFile()
        if full_text != None:
            text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=25)
            texts = text_splitter.split_documents(full_text)
            self.splitTexts = texts

    def embeddingAndVectorDB(self):
        self.splitSentences()
        embeddings = OpenAIEmbeddings()
        db = Chroma.from_documents(documents=self.splitTexts, embedding=embeddings)
        return db

    def askAndFindFiles(self, question):
        db = self.embeddingAndVectorDB()
        # retriever = db.as_retriever()
        # results = retriever.invoke(question)
        # return results

        retriever = db.as_retriever()
        llm = OpenAI(temperature=0,)
        compressor = LLMChainExtractor.from_llm(llm=llm)
        compressor_retriever = ContextualCompressionRetriever(
            base_retriever = retriever,
            base_compressor = compressor,
        )
        return compressor_retriever.get_relevant_documents(query=question)[0]
        # retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={
        #     "score_threshold":.5,'k':2
        # })
        # return retriever.get_relevant_documents(query=question)
    
    def chatWithDoc(self, question):
        _context=""
        context = self.askAndFindFiles(question)
        for i in context:
            _context += i.page_content
        messages = self.prompt.format_messages(context=_context, question=question)
        chat = ChatOpenAI(
            model='gpt-4',
            temperature=0
        )
        return chat.invoke(messages)
chat_doc = ChatDoc()
chat_doc.doc = "mri_report.docx"
chat_doc.askAndFindFiles("Age of the patient")
# chat_doc.chatWithDoc('First, tell me if this patient has an extra capsular extension or not. Second, tell me what information makes you give me this answer')

/Users/songyuze/opt/anaconda3/envs/RAG/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/songyuze/opt/anaconda3/envs/RAG/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/songyuze/opt/anaconda3/envs/RAG/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/songyuze/opt/anaconda3/envs/RAG/lib/python3.8/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document(page_content='63-year-old male', metadata={'source': 'mri_report.docx'})